In [6]:
!pip install numpy
!pip install scipy
!pip install librosa
!pip install soundfile
!pip install sounddevice
!pip install torch
!pip install tensorflow
!pip install demucs
!pip install crepe
!pip install music21
!pip install python-ly
!pip install pyguitarpro
!pip install noisereduce
!pip install musif
!pip install omnizart
!pip install onnx
!pip install onnxruntime
!pip install pydub
!pip install requests
!pip install six
!pip install tqdm
!pip install matplotlib
!pip install hmm


  Using cached omnizart-0.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl.metadata (7.8 kB)
  Using cached librosa-0.8.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached pretty_midi-0.2.10.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached pyfluidsynth-1.3.4-py3-none-any.whl.metadata (7.5 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1

In [ ]:

# Upgrade pip and core tools
!pip install --upgrade pip setuptools wheel

# Install required packages
!pip install librosa --no-build-isolation
!pip install soundfile --no-build-isolation
!pip install noisereduce --no-build-isolation
!pip install demucs --no-build-isolation
!pip install crepe --no-build-isolation
!pip install music21 --no-build-isolation
!pip install python-ly --no-build-isolation
!pip install pyguitarpro --no-build-isolation
!pip install matplotlib --no-build-isolation
!pip install tensorflow --no-build-isolation
!pip install torch --no-build-isolation
!pip install scipy --no-build-isolation
!pip install tqdm --no-build-isolation
!pip install madmom --no-build-isolation
!pip install reportlab --no-build-isolation
!pip install lxml --no-build-isolation


In [ ]:
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import required libraries
import librosa
import soundfile as sf
import noisereduce as nr
import matplotlib.pyplot as plt
from scipy import signal
from google.colab import files
import music21 as m21
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from reportlab.lib import colors
import subprocess
import tempfile
import xml.etree.ElementTree as ET
from lxml import etree

print("Upload a guitar audio file...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

def load_and_standardize_audio(file_path):
    y, sr = librosa.load(file_path, sr=44100, mono=True, duration=60)
    print(f"Loaded audio: {len(y)/sr:.2f} seconds at {sr} Hz")
    return y, sr

def pitch_detection_combined(y, sr):
    print("Detecting pitches with advanced algorithms...")

    hop_length = 512
    fmin = 65
    fmax = 1000

    pitches, magnitudes = librosa.piptrack(y=y, sr=sr, hop_length=hop_length,
                                          fmin=fmin, fmax=fmax, threshold=0.3)

    times = librosa.times_like(pitches, sr=sr, hop_length=hop_length)

    frequencies = []
    confidences = []

    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        freq = pitches[index, t]
        confidence = magnitudes[index, t]

        if freq < 75 and confidence < 0.3:
            frequencies.append(0)
            confidences.append(0)
            continue

        if confidence > 0.05:
            if 65 <= freq <= 1000:
                frequencies.append(freq)
                confidences.append(confidence)
            else:
                frequencies.append(0)
                confidences.append(0)
        else:
            frequencies.append(0)
            confidences.append(0)

    return times, np.array(frequencies), np.array(confidences)

def detect_notes_enhanced(y, sr, time, frequency, confidence):
    print("Detecting notes with advanced algorithms...")

    onset_env = librosa.onset.onset_strength(y=y, sr=sr, aggregate=np.median)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr,
                                            backtrack=True,
                                            pre_max=5, post_max=5,
                                            pre_avg=10, post_avg=10,
                                            delta=0.2, wait=30)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    filtered_onsets = []
    for i, onset in enumerate(onset_times):
        if i == 0 or onset - onset_times[i-1] >= 0.05:
            filtered_onsets.append(onset)

    onset_times = np.array(filtered_onsets)

    notes = []
    for i in range(len(onset_times)):
        start_time = onset_times[i]
        if i < len(onset_times) - 1:
            end_time = onset_times[i+1]
        else:
            end_time = len(y) / sr

        if end_time - start_time < 0.05:
            continue

        mask = (time >= start_time) & (time < end_time)
        if np.any(mask):
            freq_segment = frequency[mask]
            conf_segment = confidence[mask]

            valid_indices = (freq_segment > 0) & (conf_segment > 0.1)
            if np.any(valid_indices):
                filtered_freq = freq_segment[valid_indices]
                filtered_conf = conf_segment[valid_indices]

                if len(filtered_freq) > 0:
                    avg_freq = np.median(filtered_freq)
                    avg_conf = np.mean(filtered_conf)
                    notes.append((start_time, end_time, avg_freq, avg_conf))

    return notes

def freq_to_guitar_string_fret_optimized(freq):
    if freq <= 0:
        return None, None

    string_tunings = [40, 45, 50, 55, 59, 64]
    midi_note = round(69 + 12 * np.log2(freq / 440.0))

    best_string = None
    best_fret = None
    best_score = float('inf')

    for i, open_note in enumerate(string_tunings):
        fret = midi_note - open_note

        if 0 <= fret <= 24:
            fret_penalty = fret * 1.0

            if i >= 3 and fret > 12:
                fret_penalty *= 1.5

            string_penalty = 0
            if i == 0 or i == 5:
                string_penalty = 1

            score = fret_penalty + string_penalty

            if score < best_score:
                best_score = score
                best_string = i + 1
                best_fret = fret

    if best_string is None:
        best_string = 6
        best_fret = max(0, midi_note - 40)
        if best_fret > 24:
            best_fret = best_fret % 12

    return best_string, best_fret

def generate_tab_data(notes):
    print("Generating optimized guitar tablature data...")
    tab_data = []
    for note in notes:
        start_time, end_time, freq, conf = note
        string, fret = freq_to_guitar_string_fret_optimized(freq)
        duration = end_time - start_time

        midi_note = round(69 + 12 * np.log2(freq / 440.0))
        note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        note_idx = midi_note % 12
        octave = (midi_note // 12) - 1
        note_name = note_names[note_idx] + str(octave)

        tab_data.append({
            'start_time': start_time,
            'duration': duration,
            'string': string,
            'fret': fret,
            'note_name': note_name,
            'midi_note': midi_note,
            'confidence': conf
        })

    return tab_data

def estimate_tempo_and_meter(tab_data):
    print("Analyzing tempo and meter...")

    if not tab_data:
        return 120, (4, 4)

    sorted_notes = sorted(tab_data, key=lambda x: x['start_time'])
    iois = []

    for i in range(1, len(sorted_notes)):
        ioi = sorted_notes[i]['start_time'] - sorted_notes[i-1]['start_time']
        if 0.05 <= ioi <= 2.0:
            iois.append(ioi)

    if not iois:
        return 120, (4, 4)

    median_ioi = np.median(iois)
    bpm = int(60 / median_ioi)

    while bpm < 60:
        bpm *= 2
    while bpm > 180:
        bpm //= 2

    common_tempos = [60, 72, 80, 88, 96, 108, 120, 132, 144, 160, 176]
    tempo = min(common_tempos, key=lambda x: abs(x - bpm))

    return tempo, (4, 4)

def generate_lilypond_file(tab_data, output_path, tempo, time_sig):
    print("Generating professional LilyPond notation...")

    sorted_notes = sorted(tab_data, key=lambda x: x['start_time'])
    beat_duration = 60 / tempo

    with open(output_path, 'w') as f:
        f.write('\\version "2.22.0"\n\n')
        f.write('\\header {\n')
        f.write('  title = "Guitar Transcription"\n')
        f.write('  composer = "Auto-Transcription"\n')
        f.write('  tagline = "Transcribed with high precision algorithm"\n')
        f.write('}\n\n')

        f.write('\\paper { ragged-right = ##f }\n\n')

        f.write('music = {\n')
        f.write(f'  \\tempo {time_sig[1]} = {tempo}\n')
        f.write(f'  \\time {time_sig[0]}/{time_sig[1]}\n')
        f.write('  \\clef "treble_8"\n')

        for i, note in enumerate(sorted_notes):
            if note['note_name']:
                note_name = note['note_name'].lower()
                note_letter = note_name[0]
                note_acc = note_name[1] if len(note_name) > 1 and note_name[1] in ['#', 'b'] else ''
                note_octave = note_name[-1]

                if note_acc == '#':
                    note_acc = 'is'
                elif note_acc == 'b':
                    note_acc = 'es'

                int_octave = int(note_octave)
                octave_marker = "'" * (int_octave - 3) if int_octave > 3 else "," * (3 - int_octave)

                f.write(f' {note_letter}{note_acc}{octave_marker}4^"{note["string"]}/{note["fret"]}"')

                if (i + 1) % 4 == 0:
                    f.write(' |')

        f.write('\n}\n\n')

        f.write('\\score {\n')
        f.write('  <<\n')
        f.write('    \\new Staff \\music\n')
        f.write('    \\new TabStaff {\n')
        f.write('      \\set TabStaff.stringTunings = \\stringTuning <e\' b g d a, e,>\n')
        f.write('      \\music\n')
        f.write('    }\n')
        f.write('  >>\n')
        f.write('  \\layout { }\n')
        f.write('  \\midi { }\n')
        f.write('}\n')

    print(f"LilyPond file saved at {output_path}")
    return output_path

def generate_musicxml(tab_data, output_path, tempo, time_sig):
    print("Generating MusicXML file...")

    try:
        score = m21.stream.Score()

        score.insert(0, m21.metadata.Metadata())
        score.metadata.title = "Guitar Transcription"
        score.metadata.composer = "Auto-Transcription"

        guitar_part = m21.stream.Part()

        guitar = m21.instrument.Guitar()
        guitar_part.append(guitar)

        ts = m21.meter.TimeSignature(f"{time_sig[0]}/{time_sig[1]}")
        guitar_part.append(ts)

        tempo_mark = m21.tempo.MetronomeMark(number=tempo)
        guitar_part.append(tempo_mark)

        guitar_clef = m21.clef.TrebleClef()
        guitar_clef.octaveChange = -1
        guitar_part.append(guitar_clef)

        sorted_notes = sorted(tab_data, key=lambda x: x['start_time'])
        beat_duration = 60 / tempo

        current_measure = m21.stream.Measure(number=1)
        notes_in_measure = 0
        measure_number = 1

        for note_data in sorted_notes:
            if note_data['note_name']:
                note = m21.note.Note(note_data['note_name'])
                note.quarterLength = 1.0

                if note_data['string'] is not None and note_data['fret'] is not None:
                    tech = m21.expressions.TextExpression(f"{note_data['string']}/{note_data['fret']}")
                    note.expressions.append(tech)

                current_measure.append(note)
                notes_in_measure += 1

                if notes_in_measure >= 4:
                    guitar_part.append(current_measure)
                    measure_number += 1
                    current_measure = m21.stream.Measure(number=measure_number)
                    notes_in_measure = 0

        if notes_in_measure > 0:
            while notes_in_measure < 4:
                rest = m21.note.Rest(quarterLength=1.0)
                current_measure.append(rest)
                notes_in_measure += 1
            guitar_part.append(current_measure)

        score.insert(0, guitar_part)
        score.write('musicxml', output_path)

        print(f"MusicXML file saved at {output_path}")
        return output_path

    except Exception as e:
        print(f"Error generating MusicXML: {e}")
        return None

def generate_pdf_tablature(tab_data, output_path, tempo):
    """Generate a professional PDF tablature using reportlab"""
    print("Generating professional PDF tablature...")

    c = canvas.Canvas(output_path, pagesize=landscape(letter))
    width, height = landscape(letter)

    # Title and header
    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, height - 40, "Guitar Tablature")

    c.setFont("Helvetica", 12)
    c.drawString(50, height - 60, f"Tempo: {tempo} BPM")
    c.drawString(50, height - 80, f"Total Notes: {len(tab_data)}")

    # Tablature settings
    tab_start_y = height - 120
    string_spacing = 20
    measure_width = 120
    measures_per_line = 6
    chars_per_measure = 16

    # String names (from high E to low E)
    string_names = ['e|', 'B|', 'G|', 'D|', 'A|', 'E|']

    # Sort notes by time
    sorted_notes = sorted(tab_data, key=lambda x: x['start_time'])

    # Group notes into measures (4 beats per measure, assume quarter notes)
    beat_duration = 60 / tempo
    measure_duration = 4 * beat_duration  # 4/4 time

    measures = {}
    for note in sorted_notes:
        if note['string'] is not None and note['fret'] is not None:
            measure_idx = int(note['start_time'] / measure_duration)
            if measure_idx not in measures:
                measures[measure_idx] = []
            measures[measure_idx].append(note)

    max_measure = max(measures.keys()) if measures else 0

    # Draw tablature
    current_y = tab_start_y

    for line_start in range(0, max_measure + 1, measures_per_line):
        line_end = min(line_start + measures_per_line, max_measure + 1)

        # Draw measure numbers
        c.setFont("Helvetica", 10)
        for m_idx, measure_num in enumerate(range(line_start, line_end)):
            x_pos = 80 + m_idx * measure_width
            c.drawString(x_pos + measure_width//2 - 10, current_y + 20, f"M{measure_num + 1}")

        # Draw string lines and labels
        c.setFont("Courier", 12)
        for string_idx, string_name in enumerate(string_names):
            y_pos = current_y - string_idx * string_spacing

            # String label
            c.drawString(50, y_pos - 5, string_name)

            # Draw horizontal line for each measure in this line
            for m_idx in range(line_end - line_start):
                x_start = 80 + m_idx * measure_width
                x_end = x_start + measure_width - 10
                c.line(x_start, y_pos, x_end, y_pos)

                # Draw measure separator
                if m_idx < line_end - line_start - 1:
                    c.line(x_end, y_pos + 10, x_end, y_pos - 10)

        # Add fret numbers
        for measure_num in range(line_start, line_end):
            if measure_num in measures:
                measure_x_start = 80 + (measure_num - line_start) * measure_width

                for note in measures[measure_num]:
                    # Calculate position within measure
                    pos_in_measure = (note['start_time'] % measure_duration) / measure_duration
                    x_pos = measure_x_start + pos_in_measure * (measure_width - 10)

                    # Calculate string position
                    string_idx = note['string'] - 1  # Convert to 0-indexed
                    y_pos = current_y - string_idx * string_spacing

                    # Draw fret number
                    fret_str = str(note['fret'])
                    c.setFont("Courier-Bold", 10)
                    c.drawString(x_pos - 3, y_pos - 5, fret_str)

        # Move to next line
        current_y -= 150

        # Check if we need a new page
        if current_y < 100:
            c.showPage()
            current_y = height - 100

            # Add header to new page
            c.setFont("Helvetica-Bold", 14)
            c.drawString(50, height - 40, "Guitar Tablature (continued)")

    # Add footer with legend
    c.setFont("Helvetica", 10)
    footer_y = 30
    c.drawString(50, footer_y + 40, "Legend:")
    c.drawString(50, footer_y + 25, "• Numbers on lines indicate fret positions")
    c.drawString(50, footer_y + 10, "• Each measure represents approximately 4 beats")
    c.drawString(50, footer_y - 5, "• Strings: e(high E), B, G, D, A, E(low E)")

    c.save()
    print(f"PDF tablature saved at {output_path}")
    return output_path

def compile_lilypond_to_pdf(ly_file, output_dir):
    print("Attempting to compile LilyPond to PDF...")
    try:
        !apt-get update && apt-get install -y lilypond

        pdf_path = os.path.join(output_dir, "guitar_tab_lilypond.pdf")
        cmd = f"lilypond -o {os.path.splitext(pdf_path)[0]} {ly_file}"
        result = os.system(cmd)

        if result == 0:
            print(f"LilyPond PDF successfully generated at {pdf_path}")
            return pdf_path
        else:
            print("LilyPond compilation failed")
            return None
    except Exception as e:
        print(f"Error compiling LilyPond file: {e}")
        return None

def process_guitar_audio_high_precision(input_file, output_dir="output"):
    os.makedirs(output_dir, exist_ok=True)

    print(f"Processing file: {input_file}")
    y, sr = load_and_standardize_audio(input_file)

    print("Cleaning audio with advanced noise reduction...")
    y_reduced = nr.reduce_noise(y=y, sr=sr, stationary=False, prop_decrease=0.75)

    time, frequency, confidence = pitch_detection_combined(y_reduced, sr)

    notes = detect_notes_enhanced(y_reduced, sr, time, frequency, confidence)

    tab_data = generate_tab_data(notes)

    tempo, time_sig = estimate_tempo_and_meter(tab_data)
    print(f"Estimated tempo: {tempo} BPM")
    print(f"Estimated time signature: {time_sig[0]}/{time_sig[1]}")

    # Generate LilyPond file
    ly_path = os.path.join(output_dir, "guitar_tab.ly")
    ly_file = generate_lilypond_file(tab_data, ly_path, tempo, time_sig)

    # Generate MusicXML file
    xml_path = os.path.join(output_dir, "guitar_tab.musicxml")
    xml_file = generate_musicxml(tab_data, xml_path, tempo, time_sig)

    # Generate PDF tablature (guaranteed to work)
    pdf_path = os.path.join(output_dir, "guitar_tab.pdf")
    pdf_file = generate_pdf_tablature(tab_data, pdf_path, tempo)

    # Try to compile LilyPond to PDF as well
    lilypond_pdf = compile_lilypond_to_pdf(ly_path, output_dir)

    # Check for MIDI file
    midi_path = os.path.join(output_dir, "guitar_tab.midi")

    try:
        if ly_file:
            files.download(ly_file)
        if xml_file:
            files.download(xml_file)
        if pdf_file:
            files.download(pdf_file)
        if lilypond_pdf and lilypond_pdf != pdf_file:
            files.download(lilypond_pdf)
        if os.path.exists(midi_path):
            files.download(midi_path)
    except:
        print("Files created but couldn't be automatically downloaded.")

    return {
        "ly_file": ly_file,
        "xml_file": xml_file,
        "pdf_file": pdf_file,
        "lilypond_pdf": lilypond_pdf,
        "midi_file": midi_path if os.path.exists(midi_path) else None,
        "tab_data": tab_data
    }

# Execute the process
result = process_guitar_audio_high_precision(file_name)
print("\nProcessing complete. All output files:")
print(f"LilyPond source: {result['ly_file']}")
print(f"MusicXML file: {result['xml_file']}")
print(f"PDF tablature: {result['pdf_file']}")
if result['lilypond_pdf']:
    print(f"LilyPond PDF: {result['lilypond_pdf']}")
print(f"MIDI file: {result['midi_file']}")

print("\nInstructions for Soundslice import:")
print("1. Download the MusicXML file and the PDF file")
print("2. Create a new score in Soundslice")
print("3. Click 'Import notation' and select the MusicXML file")
print("4. If any issues occur, try importing the PDF file instead")
print("5. For best results, upload the original audio to Soundslice as well for syncing")
print("6. The PDF tablature is ready for printing or viewing")
